#### https://discuss.pytorch.org/t/how-to-give-more-importance-to-one-class/83588/5
- Al faster rcnn le va mal con los "label_opcional"

In [1]:
import sys
sys.path.append("../")

from src.training_utils.training import get_model_instance_segmentation
from src.training_utils.dataset import *
from src.constants import *
import PIL

In [2]:
import numpy as np
import cv2
from PIL import Image
from IPython.display import display
import torch
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.rpn import RPNHead

from torchvision import transforms as T

In [4]:
le_dict = get_encoder_dict(CLASSES_CSV)
num_classes = len(le_dict)+1 
num_classes

6

In [ ]:
PATH_TO_LOAD_MODEL = f"/home/nacho/TFI-Cazcarra/data/models/model_best_cardinalidades_fasterrcnn.pt"

model = torch.jit.load(PATH_TO_LOAD_MODEL)
model.to("cpu")

In [ ]:
def get_class_name(num_label, le_dict):
    reversed_le_dict = {v:k for k,v in le_dict.items()}
    return reversed_le_dict[num_label]

def draw_bbox(img, xmin, ymin, xmax, ymax, score, label): 
    txt = get_class_name(label, le_dict) + ' ' + str(score)
    img = cv2.putText(img, txt, (int(xmin), int(ymin)),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 1)

    return cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), 
                         (255,0,0), 1)

In [ ]:
model.eval()

In [ ]:
test_df = pd.read_csv(f"/home/nacho/TFI-Cazcarra/data/tiles/test_cardinalidades_linux_fixed.csv")
test_df['label_transformed'] = test_df['label'].apply(lambda x: le_dict[x])

In [ ]:
for test_path in test_df.image_path.unique()[:15]:
    image = Image.open(test_path).convert('RGB')
    tensor_image = T.ToTensor()(image)
    predictions = model([tensor_image])
#     for prediction in predictions:
#         print(predictions)
    prediction = predictions[1][0]
    for box, score, label in zip(prediction['boxes'],prediction['scores'],prediction['labels']):
        score = round(score.item(), 3)
        label = label.item()
        if score <= 0.4:
            continue
        xmin = box[0].item()
        ymin = box[1].item()
        xmax = box[2].item()
        ymax = box[3].item()
        print(xmin, ymin, xmax, ymax)
        if isinstance(image, PIL.Image.Image):
            image = draw_bbox(np.array(image), xmin, ymin, xmax, ymax, score, label)
        else:
            image = draw_bbox(image, xmin, ymin, xmax, ymax, score, label)
    display(Image.fromarray(image)) if not isinstance(image, PIL.Image.Image) else display(image)

In [6]:
import torch
a = torch.tensor((1, 2))
b = torch.tensor((1, 2))
torch.sub(a, b, alpha=1).

tensor([0, 0])

## imbalance

In [30]:
train_df = pd.read_csv(f"/home/nacho/TFI-Cazcarra/data/tiles/train_cardinalidades_linux_fixed.csv")
train_df['label_transformed'] = train_df['label'].apply(lambda x: le_dict[x])

In [32]:
train_df['class_weight'] = train_df['label'].apply(lambda x: 0.5 if "opcional" in x else 0.25)

In [33]:
train_df['class_weight'] = train_df['label'].apply(lambda x: 0.5 if "opcional" in x else 0.25)

class_weights = []
for path in sorted(train_df['image_path'].unique()):
    filtered = train_df[train_df['image_path']==path]
    class_weight = max(filtered['class_weight'].values)
    class_weights.append(class_weight)

In [35]:
sampler = WeightedRandomSampler(weights=class_weights, num_samples=3*len(train_df['image_path'].unique()), 
                                replacement=True)